In [3]:
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from collections import Counter
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from torchvision.transforms import InterpolationMode
from tqdm import tqdm
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler
import h5py
import io
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.utils import resample
from torchvision.models import efficientnet_b0, efficientnet_b7

In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, hdf5_file, transform=None, mode='train', train_data=None):
        # self.img_dir = img_dir
        self.mode = mode
        if self.mode == "train":
            print("train data init begin")
        elif self.mode == "val":
            print("val data init begin")

        self.transform = transform
        self.hdf5_file = h5py.File(hdf5_file, 'r')
        self.label_encoders = {}
        
        self.categorical_vars = ['sex', 'anatom_site_general',
                             'image_type', 'tbp_tile_type', 'tbp_lv_location', 
                             'tbp_lv_location_simple', 'attribution', 'copyright_license', 
                             'lesion_id', 'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 
                             'iddx_4', 'iddx_5', 'mel_mitotic_index']
        
        self.numerical_vars = ['age_approx', 'clin_size_long_diam_mm', 'tbp_lv_A', 
                          'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext', 'tbp_lv_C', 
                          'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 
                          'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio', 
                          'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB', 
                          'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm', 
                          'tbp_lv_eccentricity', 'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 
                          'tbp_lv_norm_border', 'tbp_lv_norm_color', 'tbp_lv_perimeterMM', 
                          'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt', 
                          'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 
                          'tbp_lv_y', 'tbp_lv_z', 'tbp_lv_dnn_lesion_confidence']

        if mode == 'train':
            self.annotations = self.get_full_dataframe(csv_file)
            self.encode_labels()
            self.normalize_numerical_data()
        elif mode == 'val' or mode == 'test' and train_data is not None:
            print("getting full dataframe")
            self.annotations = self.get_full_dataframe(csv_file, train_data=train_data)
            print("encoding labels")
            self.encode_labels(train_data)
            print("normalizing numerical data")
            self.normalize_numerical_data(train_data)


        if self.mode == "train":
            print("train data init done")
        elif self.mode == "val":
            print("val data init done")
        # print(numerical_df.isnull().sum())
        # print(categorical_df.isnull().sum())
        # print(numerical_df.values.dtype)
        # print(categorical_df.values.dtype)

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # Load image
        # img_path = os.path.join(self.img_dir, self.annotations.iloc[idx]["isic_id"] + ".jpg")
        # image = Image.open(img_path).convert("RGB")

        
        isic_id = self.annotations.iloc[idx]["isic_id"]

        if isic_id in self.hdf5_file:
            image = self.hdf5_file[isic_id]
            # Check if the data is numerical before conversion
            image_data = image[()]
            # 将字节字符串解码为图像
            image = Image.open(io.BytesIO(image_data)).convert("RGB")
                

        if self.transform:
            image = self.transform(image)

        # Load categorical data

        numerical_df = self.annotations[self.numerical_vars]
        categorical_df = self.annotations[self.categorical_vars]
        numerical_df = numerical_df.iloc[idx]
        categorical_df = categorical_df.iloc[idx]

        numerical_data = torch.tensor(numerical_df.values, dtype=torch.float)
        categorical_data = torch.tensor(categorical_df.values, dtype=torch.long)

        # Load numerical data
        if self.mode in ['train', 'val']:
            label = torch.tensor(int(self.annotations.iloc[idx]["target"]))
            return image, categorical_data, numerical_data, label

        elif self.mode == 'test':
            isic_id = self.annotations.iloc[idx]["isic_id"]
            return image, categorical_data, numerical_data, isic_id
        

    def get_full_dataframe(self, df, train_data=None):
        def fill_missing_with_distribution(series, distribution):
            missing_indices = series[series.isna()].index
            filled_values = np.random.choice(distribution.index, size=len(missing_indices), p=distribution.values)
            series.loc[missing_indices] = filled_values
            return series
        
        if self.mode == 'train':
            df['lesion_id'] = df['lesion_id'].apply(lambda x: 1 if pd.notnull(x) else 0)
        elif self.mode != 'train':
            for category in ['lesion_id', 'mel_mitotic_index']:
                dis = train_data.annotations[category].value_counts(normalize=True)
                generated_lesion_ids = np.random.choice(dis.index, size=len(df), p=dis.values)
                df[category] = generated_lesion_ids

        for category in ['sex', 'anatom_site_general']:
            dis = df[category].value_counts(normalize=True)
            df[category] = fill_missing_with_distribution(df[category], dis)
        
        mean_age = df['age_approx'].mean()
        df['age_approx'] = df['age_approx'].fillna(mean_age)
        
        if self.mode == 'train':
            df['iddx_2'] = df['iddx_2'].fillna(df['iddx_1'])
            df['iddx_3'] = df['iddx_3'].fillna(df['iddx_2'])
            df['iddx_4'] = df['iddx_4'].fillna(df['iddx_3'])
            df['iddx_5'] = df['iddx_5'].fillna(df['iddx_4'])
        elif self.mode != 'train':
            dis = train_data.annotations['iddx_full'].value_counts(normalize=True)
            generated_lesion_ids = np.random.choice(dis.index, size=len(df), p=dis.values)
            df['iddx_full'] = generated_lesion_ids
            for d in ['iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5']:
                df[d] = df['iddx_full']
        if self.mode != 'train':
            tbp_lv_dnn_lesion_confidence_mean = train_data.annotations['tbp_lv_dnn_lesion_confidence'].mean()
            tbp_lv_dnn_lesion_confidence_std = train_data.annotations['tbp_lv_dnn_lesion_confidence'].std()

            df['tbp_lv_dnn_lesion_confidence'] = np.random.normal(loc=tbp_lv_dnn_lesion_confidence_mean, scale=tbp_lv_dnn_lesion_confidence_std, size = len(df))

        return df
    
    def encode_labels(self, train_data=None):
        if self.mode == 'train':
            for col in self.categorical_vars:
                le = LabelEncoder()
                self.annotations[col] = le.fit_transform(self.annotations[col])
                self.label_encoders[col] = le
        elif self.mode != 'train' and train_data is not None:
            for col in self.categorical_vars:
                le = train_data.label_encoders[col]
                mask = ~self.annotations[col].isin(le.classes_)
    
                # 对新标签进行随机替换
                if mask.any():
                    self.annotations.loc[mask, col] = np.random.choice(
                        le.classes_, size=mask.sum(), p=train_data.annotations[col].value_counts(normalize=True).values
                    )
                
                # 进行编码
                self.annotations[col] = le.transform(self.annotations[col])
                print(f"Label encoder for {col} has {len(le.classes_)} classes")

                # self.annotations[col] = self.annotations[col].apply(
                #     lambda x: x if x in le.classes_ else np.random.choice(le.classes_, p=train_data.annotations[col].value_counts(normalize=True).values)
                # )
                # self.annotations[col] = le.transform(self.annotations[col])
                # print(f"Label encoder for {col} has {len(le.classes_)} classes")

    def normalize_numerical_data(self, train_data=None):
        if self.mode == 'train':
            scaler = MinMaxScaler()
            self.annotations[self.numerical_vars] = scaler.fit_transform(self.annotations[self.numerical_vars])
        elif self.mode != 'train' and train_data is not None:
            scaler = MinMaxScaler()
            scaler.fit(train_data.annotations[train_data.numerical_vars])
            self.annotations[self.numerical_vars] = scaler.transform(self.annotations[self.numerical_vars])

In [5]:
def get_transform():
    transform = transforms.Compose([
        transforms.Resize(256, interpolation=InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform

In [6]:
train_df = pd.read_csv("../../data/train-metadata.csv")
mytransform = get_transform()
train_set = CustomImageDataset(csv_file=train_df, hdf5_file="../../data/train-image.hdf5", transform=mytransform)

/tmp/ipykernel_4137629/2728867995.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("../../data/train-metadata.csv")


train data init begin


/tmp/ipykernel_4137629/2876828260.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series.loc[missing_indices] = filled_values
/tmp/ipykernel_4137629/2876828260.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series.loc[missing_indices] = filled_values


train data init done


In [7]:
df = train_set.annotations.copy()

In [ ]:
df.describe()

In [ ]:
df.info()

In [12]:
df_mag = df[df['target'] == 1].copy(deep = True)

In [ ]:
df_mag.info()

In [9]:
X = df.drop(columns=['target', 'isic_id', 'patient_id', 'mel_thick_mm'])


In [10]:
X_np = df.drop(columns=['target', 'isic_id', 'patient_id', 'mel_thick_mm']).to_numpy()
y_np = df['target'].to_numpy()

In [12]:
categorical_vars = ['sex', 'anatom_site_general', 'image_type', 'tbp_tile_type', 'tbp_lv_location', 
                                 'tbp_lv_location_simple', 'attribution', 'copyright_license', 'lesion_id', 
                                 'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5', 'mel_mitotic_index']

numerical_vars = ['age_approx', 'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 
                               'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 
                               'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 
                               'tbp_lv_deltaA', 'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm', 
                               'tbp_lv_eccentricity', 'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 
                               'tbp_lv_norm_color', 'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 
                               'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y', 
                               'tbp_lv_z', 'tbp_lv_dnn_lesion_confidence']

len(categorical_vars) + len(numerical_vars)

51

In [13]:
categorical_indices = [X.columns.get_loc(col) for col in categorical_vars if col in X.columns]
# categorical_indices = [ 1 if col in categorical_vars  else 0  for  col in df.columns]

categorical_indices


[1, 2, 4, 5, 25, 26, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]

In [19]:
print(f"Shape of X before resampling: {X.shape}")
print(f"Shape of y before resampling: {y.shape}")


Shape of X before resampling: (401059, 51)
Shape of y before resampling: (401059,)


In [14]:
from imblearn.over_sampling import SMOTENC

# 假设你希望少数类样本占总样本的 20%
sampling_strategy = 0.2

smote_nc = SMOTENC(categorical_features=categorical_indices, sampling_strategy=sampling_strategy, random_state=42)
X_res, y_res = smote_nc.fit_resample(X_np, y_np)

print(f'Resampled dataset shape: {Counter(y_res)}')

Resampled dataset shape: Counter({np.int64(0): 400666, np.int64(1): 80133})


In [22]:
X_res.shape


(480799, 51)

In [16]:
mask = y_res == 1

# 2. 提取 X_res 中对应的样本
X_res_minority = X_res[mask]

# 3. 提取 y_res 中等于 1 的部分
y_res_minority = y_res[mask]

In [15]:
Counter(y_res)

Counter({np.int64(0): 400666, np.int64(1): 80133})

In [18]:
import pandas as pd
import numpy as np

# 假设 X_res 是 (n_samples, n_features) 的 ndarray
# 假设 y_res 是 (n_samples,) 的 ndarray

# 1. 将 X_res 转换为 DataFrame
X_res_df = pd.DataFrame(X_res_minority[:10100], columns=X.columns)

# 2. 将 y_res 添加为目标列
X_res_df['target'] = y_res_minority[:10100]

# 3. 保存 DataFrame 为 CSV 文件
output_file = 'resampled_data.csv'
X_res_df.to_csv(output_file, index=False)

print(f"Resampled data saved to {output_file}")


Resampled data saved to resampled_data.csv


In [1]:
from collections import Counter
from numpy.random import RandomState
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTENC
X, y = make_classification(n_classes=2, class_sep=2,
weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0,
n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)
print(f'Original dataset shape {X.shape}')
print(f'Original dataset samples per class {Counter(y)}')
# simulate the 2 last columns to be categorical features
X[:, -2:] = RandomState(10).randint(0, 4, size=(1000, 2))
sm = SMOTENC(random_state=42, categorical_features=[18, 19])
X_res, y_res = sm.fit_resample(X, y)
print(f'Resampled dataset samples per class {Counter(y_res)}')

Original dataset shape (1000, 20)
Original dataset samples per class Counter({np.int64(1): 900, np.int64(0): 100})
Resampled dataset samples per class Counter({np.int64(0): 900, np.int64(1): 900})


In [2]:
print(f"Shape of X after resampling: {X_res.shape}")

Shape of X after resampling: (1800, 20)
